# Downloader and playlist creator of songs from Tracks

Step 1: Run `tracks-downloader.py` to get all tracks from the website. Results in 24148 entries in `tracks-db.txt`.

Step 2: Run `tracks-db-remove-duplicates.py` to create `tracks-db-no-duplicates.txt` with 6805 entries.